In [10]:
import os

In [14]:
os.chdir("Image-classification-DL")

In [29]:
%pwd

'/Volumes/Personal/projects/vision/Image-classification-DL'

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [31]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml,create_directories
import tensorflow as tf


In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_training_config(self)->TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")
        create_directories([Path(training.root_dir)])
        
        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config        

In [33]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [34]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [35]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-12-20 18:32:31,589: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-20 18:32:31,591: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-20 18:32:31,592: INFO: common: created directory at: artifacts]
[2024-12-20 18:32:31,592: INFO: common: created directory at: artifacts/training]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
Epoch 1/40


2024-12-20 18:32:32.810987: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


99/99 [==============================] - ETA: 0s - loss: 4.2201 - accuracy: 0.6289

2024-12-20 18:33:26.038313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


99/99 [==============================] - 65s 639ms/step - loss: 4.2201 - accuracy: 0.6289 - val_loss: 2.6380 - val_accuracy: 0.4533
Epoch 2/40
99/99 [==============================] - 67s 672ms/step - loss: 1.0940 - accuracy: 0.7592 - val_loss: 2.5740 - val_accuracy: 0.5713
Epoch 3/40
99/99 [==============================] - 65s 654ms/step - loss: 0.8724 - accuracy: 0.8017 - val_loss: 2.9461 - val_accuracy: 0.5642
Epoch 4/40
99/99 [==============================] - 65s 651ms/step - loss: 0.7326 - accuracy: 0.8428 - val_loss: 2.9595 - val_accuracy: 0.5671
Epoch 5/40
99/99 [==============================] - 65s 649ms/step - loss: 0.9850 - accuracy: 0.8170 - val_loss: 2.9586 - val_accuracy: 0.6500
Epoch 6/40
99/99 [==============================] - 65s 651ms/step - loss: 0.6528 - accuracy: 0.8654 - val_loss: 3.3770 - val_accuracy: 0.6237
Epoch 7/40
99/99 [==============================] - 65s 654ms/step - loss: 0.5783 - accuracy: 0.8782 - val_loss: 2.9302 - val_accuracy: 0.6717
Epoch 8/40